In [64]:
from tensorflow import keras

In [67]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels),(test_images,test_labels) = mnist.load_data()

11493376/11490434 [==============================] - 8s 1us/step


(60000, 28, 28)

In [69]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import examples.utils as utils
import time
import os

In [70]:
mnist_path = 'data/mnist'
utils.download_mnist(mnist_path)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


In [71]:
learning_rate = 0.01
batch_size = 128
epochs = 30
n_train = 60000
n_test = 10000

In [72]:
train,val,test = utils.read_mnist(mnist_path, flatten = True)

In [77]:
tf.reset_default_graph()
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(1000)
test_data = tf.data.Dataset.from_tensor_slices(test)

print("train data output types", train_data.output_types)
print("train data output shapes", train_data.output_shapes)

train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img,label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)

w = tf.get_variable(name = "weights", shape = (784,10), initializer = tf.random_normal_initializer(0,0.01))
b = tf.get_variable(name = "bias",shape = (1,10))

logits = tf.matmul(img,w) + b
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = logits, name = 'entropy')
loss = tf.reduce_mean(entropy)

opt = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)


preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds,1), tf.argmax(label,1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

train data output types (tf.float32, tf.float64)
train data output shapes (TensorShape([Dimension(784)]), TensorShape([Dimension(10)]))


In [78]:
with tf.Session() as sess:
    time = time.time()
    sess.run(tf.global_variables_initializer())
    
    
    for i in range(epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        
        try:
            while True:
                _,l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
                pass
            
        print("Epoch:{0}, Avg Loss:{1}".format(i, total_loss/n_batches))
    print("Total time", time - time.time())

    
    sess.run(train_init)
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy
    except tf.errors.OutOfRangeError:
        pass
    
    
    print("Training Accuracy", correct_preds/n_train)
        
        

ValueError: Fetch argument <tf.Operation 'GradientDescent' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_weights/ApplyGradientDescent"
input: "^GradientDescent/update_bias/ApplyGradientDescent"
 is not an element of this graph.)